In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
# Doublet candidates
cat_doublet = Table.read('../catalogs/COLA1_O3_fitted_flux.fits').to_pandas()
cat_doublet['NUMBER'] = cat_doublet['NUMBER_1']
cat_doublet = cat_doublet.drop(columns=['NUMBER_1'])
# Singlet candidates
cat_singlet = Table.read('../catalogs/COLA1_O3_fitted_flux_singlet.fits').to_pandas()

print(cat_doublet.dtypes)
print(cat_singlet.dtypes)

In [ ]:
# Join singlet and doublet catalogs
cat = cat_doublet.merge(cat_singlet, left_on='NUMBER', right_on='NUMBER',
                        how='outer', suffixes=['_d', '_s'])

def calculate_z(row):
    if pd.notna(row['z_O3_5008']) and pd.notna(row['z_O3doublet']):
        return (row['z_O3_5008'] + row['z_O3doublet']) * 0.5
    else:
        return row['z_O3_5008'] if pd.notna(row['z_O3_5008']) else row['z_O3doublet']

def calculate_RA(row):
    if pd.notna(row['RA']) and pd.notna(row['ALPHA_J2000_det']):
        return (row['RA'] + row['ALPHA_J2000_det']) * 0.5
    else:
        return row['RA'] if pd.notna(row['RA']) else row['ALPHA_J2000_det']

def calculate_DEC(row):
    if pd.notna(row['DEC']) and pd.notna(row['DELTA_J2000_det']):
        return (row['DEC'] + row['DELTA_J2000_det']) * 0.5
    else:
        return row['DEC'] if pd.notna(row['DEC']) else row['DELTA_J2000_det']

cat['z'] = cat.apply(calculate_z, axis=1)
cat['RA'] = cat.apply(calculate_RA, axis=1)
cat['DEC'] = cat.apply(calculate_DEC, axis=1)

cat['det_doublet'] = (cat['z_O3doublet'] > 0)
cat['det_singlet'] = (cat['z_O3_5008'] > 0)

cat.dtypes

In [ ]:
fig, ax = plt.subplots()

ax.hist(cat['z'], bins=50)
ax.set_xlabel('z')
ax.axvline(5.877)

plt.show()

fig, ax = plt.subplots()

zbins = np.linspace(6.45, 6.65, 40)
ax.hist(cat['z'], bins=zbins, label='all', alpha=0.3)
ax.hist(cat['z'][cat['det_doublet']], bins=zbins, label='doublet', histtype='step',
        lw=2)
ax.hist(cat['z'][cat['det_singlet']], bins=zbins, label='singlet', histtype='step',
        lw=2)
ax.set_xlabel('z')

ax.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.scatter('RA', 'DEC', data=cat[cat['det_doublet']],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
cbar = ax.scatter('RA', 'DEC', data=cat[cat['det_singlet']],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 9269],
           marker='*', s=300, c='k')

fig.colorbar(cbar)

ax.set_aspect('equal')

ax.set_xlim(150.6, 150.7)
ax.set_ylim(2.135, 2.27)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

mask = (cat['z'] > 6.45) & (cat['z'] < 6.65)

z_min = cat['z'][mask].min()
z_max = cat['z'][mask].max()

ax.scatter('RA', 'DEC', data=cat[cat['det_doublet'] & mask],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
cbar = ax.scatter('RA', 'DEC', data=cat[cat['det_singlet'] & mask],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 9269],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# Distance at z=5.877
ang_diameter_dist = cosmo.angular_diameter_distance(5.877) / u.rad
print(ang_diameter_dist)
ang_diameter_dist = ang_diameter_dist.to(u.kpc / u.deg)
ang_per_dist = (300 * u.kpc / ang_diameter_dist)
ax.errorbar(150.67, 2.16, xerr=ang_per_dist.value * 0.5,
            fmt='none', capsize=4, c='k', elinewidth=2,
            capthick=2)
ax.text(150.67, 2.156, '300 pkpc',
        horizontalalignment='center')

fig.colorbar(cbar)
ax.set_title('6.45 < z < 6.65')

ax.set_aspect('equal')

ax.set_xlim(150.6, 150.7)
ax.set_ylim(2.135, 2.27)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

mask = (cat['z'] > 5.877 - 0.077) & (cat['z'] < 5.877 + 0.077)

z_min = cat['z'][mask].min()
z_max = cat['z'][mask].max()

ax.scatter('RA', 'DEC', data=cat[cat['det_doublet'] & mask],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
cbar = ax.scatter('RA', 'DEC', data=cat[cat['det_singlet'] & mask],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1's clone
ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 9275],
           marker='*', s=300, c='z', cmap='gist_rainbow', vmin=z_min, vmax=z_max)
ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 6123],
           marker='*', s=300, c='z', cmap='gist_rainbow', vmin=z_min, vmax=z_max)
# ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 3590],
#            marker='*', s=300, c='z', cmap='gist_rainbow', vmin=z_min, vmax=z_max)
# ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 4382],
#            marker='*', s=300, c='z', cmap='gist_rainbow', vmin=z_min, vmax=z_max)

# Distance at z=5.877
ang_diameter_dist = cosmo.angular_diameter_distance(5.877) / u.rad
print(ang_diameter_dist)
ang_diameter_dist = ang_diameter_dist.to(u.kpc / u.deg)
ang_per_dist = (1000 * u.kpc / ang_diameter_dist)
ax.errorbar(150.67, 2.16, xerr=ang_per_dist.value * 0.5,
            fmt='none', capsize=4, c='k', elinewidth=2,
            capthick=2)
ax.text(150.67, 2.156, '1 pMpc',
        horizontalalignment='center')

fig.colorbar(cbar)
ax.set_title('5.65 < z < 5.95')

ax.set_aspect('equal')

ax.set_xlim(150.6, 150.7)
ax.set_ylim(2.135, 2.27)

plt.show()

In [ ]:
# Distance from each source to COLA1 in the comoving frame
COLA1_index = (cat['NUMBER'] == 9269)

cdist0 = cosmo.comoving_distance(cat['z'])
COLA1_dist = cosmo.comoving_distance(cat['z'][COLA1_index])

coord_Arr = SkyCoord(np.array(cat['RA']) * u.deg,
                     np.array(cat['DEC']) * u.deg,
                     distance=cdist0)
coord_COLA1 = SkyCoord(np.array(cat['RA'][COLA1_index]) * u.deg,
                       np.array(cat['DEC'][COLA1_index]) * u.deg,
                       distance=COLA1_dist)

dist_to_COLA1 = coord_COLA1.separation_3d(coord_Arr).to(u.Mpc)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

z_min = cat['z'].min()
z_max = cat['z'].max()

mask_bad_fit = (cat['f_O3_5008_err_d'] < 90.) & (cat['f_O3_4960_err_d'] < 90.)
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[mask_bad_fit],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
mask_bad_fit = (cat['f_O3_5008_err_s'] < 90.) & (cat['f_O3_4960_err_s'] < 90.)
cbar = ax.scatter('f_O3_5008_s', 'f_O3_4960_s', data=cat[mask],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 9269],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 9275],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 6123],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# TMP!!!!!!!!!!
ax.set_ylim(-1, 20)

fig.colorbar(cbar)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

mask = (cat['z'] > 6.45) & (cat['z'] < 6.65)

z_min = cat['z'][mask].min()
z_max = cat['z'][mask].max()

mask_bad_fit = (cat['f_O3_5008_err_d'] < 90.) & (cat['f_O3_4960_err_d'] < 90.) & mask
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[mask_bad_fit],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
mask_bad_fit = (cat['f_O3_5008_err_s'] < 90.) & (cat['f_O3_4960_err_s'] < 90.) & mask
cbar = ax.scatter('f_O3_5008_s', 'f_O3_4960_s', data=cat[mask],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 9269],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# TMP!!!!!!!!!!
ax.set_ylim(-1, 20)

ax.set_title('6.45 < z < 6.65')

fig.colorbar(cbar)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

mask = (cat['z'] > 5.877 - 0.077) & (cat['z'] < 5.877 + 0.077)

z_min = cat['z'][mask].min()
z_max = cat['z'][mask].max()

mask_bad_fit = (cat['f_O3_5008_err_d'] < 90.) & (cat['f_O3_4960_err_d'] < 90.) & mask
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[mask_bad_fit],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 9275],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 6123],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# TMP!!!!!!!!!!
ax.set_ylim(-1, 20)

ax.set_title('5.80 < z < 5.95')

fig.colorbar(cbar)

plt.show()

In [ ]:
Hbeta_SN = cat['f_Hb_d'] / cat['f_Hb_err_d']
O3_5008_SN = cat['f_O3_5008_d'] / cat['f_O3_5008_err_d']
mask_Hbeta_SN = (Hbeta_SN > 3) & (O3_5008_SN > 5) & (cat['F115W_AUTO_mag'] < 90)


fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

cbar = ax.scatter(cat['F115W_AUTO_mag'][mask_Hbeta_SN],
                 (cat['f_O3_5008_d'] / cat['f_Hb_d'])[mask_Hbeta_SN],
                 c=cat['z'][mask_Hbeta_SN], cmap='rainbow', vmin=z_min, vmax=z_max)
ax.scatter(cat['F115W_AUTO_mag'][cat['NUMBER'] == 9269],
           (cat['f_O3_5008_d'] / cat['f_Hb_d'])[cat['NUMBER'] == 9269],
           marker='*', s=200,
           c=cat['z'][cat['NUMBER'] == 9269], cmap='rainbow', vmin=z_min, vmax=z_max)

fig.colorbar(cbar)

ax.set_title('O3_5008 / Hbeta ratio')

ax.set_xlabel('F115W [mag]')
ax.set_ylabel('f_O3_5008 / f_Hbeta')

plt.show()

In [ ]:
fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.errorbar('f_O3_5008_d', 'f_Hb_d',
            xerr='f_O3_5008_err_d',
            yerr='f_Hb_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k')

ax.errorbar('f_O3_5008_d', 'f_Hb_d',
            xerr='f_O3_5008_err_d',
            yerr='f_Hb_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15)

ax.set_xlabel('f_O3_5008')
ax.set_ylabel('f_Hbeta')

plt.show()

In [ ]:
fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.errorbar('F115W_AUTO_mag', 'f_Hb_d',
            yerr='f_Hb_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k')
ax.errorbar('F115W_AUTO_mag', 'f_Hb_d',
            yerr='f_Hb_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15)

ax.set_xlabel('F115W')
ax.set_ylabel('f_Hbeta')

plt.show()

fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.errorbar('F115W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k')
ax.errorbar('F115W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15)

ax.set_xlabel('F115W')
ax.set_ylabel('f_O3_5008')

plt.show()

In [ ]:
# Load eiger catalogs
eiger_cat_list = ['J0100_photcat_v4_O3CANDIDATES_all_CLUMPS.fits',  'J1030_photcat_v1_O3CANDIDATES_all_CLUMPS.fits', 
                  'J1148_photcat_v2_O3CANDIDATES_all_CLUMPS.fits', 'J0148_photcat_v1_O3CANDIDATES_all_CLUMPS.fits', 
                  'J1120_photcat_v2_O3CANDIDATES_all_CLUMPS.fits']
eiger_cat_dir = '/home/alberto/cosmos/ista/COLA1/catalogs/eiger'


tab_list = []

for cat_name in eiger_cat_list:
    tab_list.append(Table.read(f'{eiger_cat_dir}/{cat_name}').to_pandas())

eiger_cat = pd.concat(tab_list)

In [ ]:
fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.scatter('f_O3_5008', 'f_Hb', data=eiger_cat, s=2,
           label='EIGER')

ax.errorbar('f_O3_5008_d', 'f_Hb_d',
            xerr='f_O3_5008_err_d',
            yerr='f_Hb_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k', label='COLA1 (all sources)')

ax.errorbar('f_O3_5008_d', 'f_Hb_d',
            xerr='f_O3_5008_err_d',
            yerr='f_Hb_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15, label='COLA1')

ax.legend()

ax.set_xlabel('f_O3_5008')
ax.set_ylabel('f_Hbeta')

ax.set_ylim(-1, 10)
ax.set_xlim(-1, 40)

plt.show()

In [ ]:
fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.scatter(eiger_cat['MAG_AUTO_F115W_apcor'], eiger_cat['f_Hb'], s=2,
           label='EIGER')

ax.errorbar('F115W_AUTO_mag', 'f_Hb_d',
            yerr='f_Hb_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k', label='COLA1 (all sources)')
ax.errorbar('F115W_AUTO_mag', 'f_Hb_d',
            yerr='f_Hb_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15, label='COLA1')

ax.legend()

ax.set_xlabel('F115W')
ax.set_ylabel('f_Hbeta')

ax.set_ylim(-1, 10)
ax.set_xlim(23, 30.5)

plt.show()

fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.scatter(eiger_cat['MAG_AUTO_F115W_apcor'], eiger_cat['f_O3_5008'], s=2,
           label='EIGER')

ax.errorbar('F115W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k', label='COLA1 (all sources)')
ax.errorbar('F115W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15, label='COLA1')

ax.set_xlabel('F115W')
ax.set_ylabel('f_O3_5008')

ax.legend()

ax.set_ylim(-1, 40)
ax.set_xlim(23, 30.5)

plt.show()